- Define functions for data processing
- User setup section
- Load preprocessed data for assigning weather types
- Calculate ERA5 normalization data  (NOTE: reverse ERA5. Original ERA5 is from north to South)
- Associate data to WTs
- Plot the ensemble mean WT frequency during the NAM season

In [14]:
import datetime
from datetime import datetime
import glob
from netCDF4 import Dataset
import sys, traceback
import dateutil.parser as dparser
import string
from pdb import set_trace as stop
import numpy as np
import os
import mpl_toolkits
import pickle
import subprocess
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib as mpl
# import pylab as plt
import scipy
import matplotlib.path as mplPath
from matplotlib.patches import Polygon as Polygon2
from scipy.ndimage import gaussian_filter
import seaborn as sns
import sys
import scipy.spatial.qhull as qhull
import xarray as xr
import cfgrib
from pandas.tseries.offsets import MonthEnd

#### Define functions for data processing

In [15]:
# Interpolation functions
def interp_weights(xy, uv,d=2):
    tri = qhull.Delaunay(xy)
    simplex = tri.find_simplex(uv)
    vertices = np.take(tri.simplices, simplex, axis=0)
    temp = np.take(tri.transform, simplex, axis=0)
    delta = uv - temp[:, d]
    bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
    return vertices, np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))

def interpolate(values, vtx, wts):
    return np.einsum('nj,nj->n', np.take(values, vtx), wts)

# ###################################################
# ###################################################
def PreprocessWTdata(DailyVarsInput,                 # WT data [time,lat,lon,var]
                     RelAnnom=1,                     # calculate relative anomalies [1-yes; 0-no]
                     SmoothSigma=0,                  # Smoothing stddev (Gaussian smoothing)
                     RemoveAnnualCycl=1,             # remove annual cycle [1-yes; 0-no]
                     NormalizeData='D',              # normalize variables | options are  - 'C' - climatology
                                                     # - 'D' - daily (default)
                                                     # - 'N' - none
                     ReferencePer=None,              # period for normalizing the data
                     Normalize = None):              # mean, std, and spatial mean for normalization
    

    DailyVars = np.copy(DailyVarsInput)
    # Calculate relative anomaly
    if RelAnnom == 1:
        # we have to work with absolute values for this since we risk to divide by zero values in the climatology
        DailyVars=np.abs(DailyVars)
        if Normalize is None:
            if ReferencePer is None:
                DailyVars=(DailyVars-np.mean(DailyVars, axis=0)[None,:])/np.mean(DailyVars, axis=0)[None,:]
            else:
                DailyVars=(DailyVars-np.mean(DailyVars[ReferencePer], axis=0)[None,:])/np.mean(DailyVars[ReferencePer], axis=0)[None,:]
        else:
            # calculate anomalies with provided climatology
            DailyVars=(DailyVars - Normalize[2][None,:])/Normalize[2][None,:]

    if len(DailyVars.shape) == 3:
        DailyVars = DailyVars[:,:,:,None]
    # Spatially smooth the data
    DailyVars=gaussian_filter(DailyVars[:,:,:,:], sigma=(0,SmoothSigma,SmoothSigma,0))

    # Remove the annual cycle
    if RemoveAnnualCycl == 1:
        SpatialMeanData=pd.DataFrame(np.nanmean(DailyVars, axis=(1,2)))
        Averaged=np.roll(np.array(SpatialMeanData.rolling(window=21).mean()), -10, axis=0)
        Averaged[:10,:]=Averaged[11,:][None,:]; Averaged[-10:,:]=Averaged[-11,:][None,:]
        DailyVars=DailyVars-Averaged[:,None,None,:]

    # Normalize the data
    if NormalizeData == 'D':
        DailyVars=(DailyVars-np.mean(DailyVars, axis=(1,2))[:,None,None,:])/np.std(DailyVars, axis=(1,2))[:,None,None,:]
    elif NormalizeData == 'C':
        if Normalize is None:
            if ReferencePer is None:
                DailyVars=(DailyVars-np.mean(DailyVars, axis=(0,1,2))[None,None,None,:])/np.std(DailyVars, axis=(0,1,2))[None,None,None,:]
            else:
                DailyVars=(DailyVars-np.mean(DailyVars[ReferencePer], axis=(0,1,2))[None,None,None,:])/np.std(DailyVars[ReferencePer], axis=(0,1,2))[None,None,None,:]
        else:
            # use predefined normalization terms
            DailyVars=((DailyVars - Normalize[0][None,None,None,:]))/Normalize[1][None,None,None,:]
        DailyVars[np.isnan(DailyVars)]=0

    return DailyVars

# ===================================================================
def EucledianDistance(DailyVars,
                      rgrClustersFin,
                      MoreDistances=0):  # if this key is 1 the function will calculate additional distance metrics
    from scipy.spatial import distance
    
    SHAPE=DailyVars.shape
    Data_flatten=np.reshape(DailyVars, (SHAPE[0],SHAPE[1]*SHAPE[2]*SHAPE[3]))
    EucledianDist=np.zeros((SHAPE[0],rgrClustersFin[0].shape[0])); EucledianDist[:]=np.nan
    Correlation=np.copy(EucledianDist)
    Manhattan=np.copy(EucledianDist)
    Chebyshev=np.copy(EucledianDist)
    for dd in range(SHAPE[0]):
        EucledianDist[dd,:] = np.array([np.linalg.norm(rgrClustersFin[0][wt,:]-Data_flatten[dd,:]) for wt in range(rgrClustersFin[0].shape[0])])
        Correlation[dd,:] = np.array([np.corrcoef(rgrClustersFin[0][wt,:],Data_flatten[dd,:])[0][1] for wt in range(rgrClustersFin[0].shape[0])])
        
        if MoreDistances == 1:
            for wt in range(rgrClustersFin[0].shape[0]):
                x = Data_flatten[dd,:] #rgrClustersFin[0][wt,:]
                YY = rgrClustersFin[0][wt,:] #+np.random.rand(len(rgrClustersFin[0][wt,:]))
                XX = Data_flatten[dd,:]

                # ----- Manhattan Distance ------
                # Quoting from the paper, “On the Surprising Behavior of Distance Metrics in High Dimensional Space”, by Charu C. Aggarwal, Alexander 
                # Hinneburg, and Daniel A. Kiem. “ for a given problem with a fixed (high) value of the dimensionality d, it may be preferable to use 
                # lower values of p. This means that the L1 distance metric (Manhattan Distance metric) is the most preferable for high dimensional applications.”
                Manhattan[dd,wt] = distance.cityblock(XX, YY)
                Chebyshev[dd,wt] = distance.chebyshev(XX, YY)
    
    return EucledianDist, Correlation, Manhattan, Chebyshev

In [16]:
def grNormalization(data_4d, vtx, wts, n_lat, n_lon):
    grNormalizationFactors = {}
    Mean = np.nanmean(data_4d, axis=(0,1))

    # interpolate to centroid grid   
    Mean_wt = interpolate(Mean.flatten(), vtx, wts).reshape(n_lat, n_lon) 

    # reshape Mean by adding back axis=(1,2) because of np.nanmean
    Anomalies = (data_4d - Mean[None,None,:,:])/Mean[None,None,:,:]

    # Convert input scalar to 1d array with at least one dimension
    # in order to call function  PreprocessWTdata
    Normalize = [np.atleast_1d(np.nanmean(Anomalies)), np.atleast_1d(np.nanstd(Anomalies)), Mean_wt[:,:,None]]
    grNormalizationFactors[0] = Normalize
    print('Amomalies', np.nanmean(Anomalies))
    print(flnm_n)
    np.savez(flnm_n, grNormalizationFactors = grNormalizationFactors)
    #NormData = np.load(flnm_n, allow_pickle=True)
    # NormData = NormData['grNormalizationFactors']    

#### User setup section

In [39]:
RegName = 'Arizona West' # can be 'Arizona West', 'Arizona East', 'NM North', and 'NM South'
#RegName = 'Arizona East'
#RegName = 'NM North'
#RegName = 'NM South'
MONTHS  = [6,7,8,9,10]    # months within the NAM season
month_day_dict = {6: 30, 7:31, 8:31, 9:30, 10:31}

month_day_dict[10] # [MONTHS[0]]

31

#### Load preprocessed data for assigning weather types

In [40]:
# Setup clustering algorithm
ClusterMeth='HandK'  # current options are ['HandK','hdbscan']
ClusterBreakup = 0   # breakes up clusters that are unproportionally large (only for hdbscan)
RelAnnom=1           # 1 - calculates daily relative anomalies
NormalizeData='C'    # normalize variables | options are  - 'C' - climatology
                                                        # - 'D' - daily (default)
                                                        # - 'N' - none
MinDistDD=1          # minimum nr of days between XWT events
RemoveAnnualCycl=0   # remove annual cycle in varaiables with 21 day moving average filter

dir_data = '/glade/campaign/mmm/c3we/mingge/COEXIST/AZ_S2S_DATA/'
if RegName == 'Arizona West':
    # cluster 1501
    sCentroidFile = dir_data + '1501_XWT-centroids_train-1982-2018_eval-1982-2018_E13514_XWTs3_Vars-Q850_M-6-7-8-9-10.nc'
    sERA_data = dir_data + 'ERA-Interim_PRISM_data13514_1501_1982-2018_Q850_JJASO.npz'
    sWTdata = dir_data + 'Clusters13514_1501_1982-2018_Q850_JJASO'
    NormFact = dir_data + 'Normalization_Factors_IFS_1501.npz'
    WTsort = [0,2,1]
    re = 0
if RegName == 'Arizona East':
    # cluster 1502
    sERA_data = dir_data + 'ERA-Interim_PRISM_data13514_1502_1982-2018_Q850_JJASO.npz'
    sCentroidFile = dir_data +'1502_XWT-centroids_train-1982-2018_eval-1982-2018_E13514_XWTs3_Vars-Q850_M-6-7-8-9-10.nc'
    sWTdata = dir_data + 'Clusters13514_1502_1982-2018_Q850_JJASO'
    NormFact = dir_data + 'Normalization_Factors_IFS_1502.npz'
    WTsort = [1,2,0]
    re = 1
if RegName == 'NM North':
    # cluster HUC6-00
    sERA_data = dir_data + 'ERA-Interim_PRISM_data13514_HUC6-00_1982-2018_Q850_JJASO.npz'
    sCentroidFile = dir_data + 'HUC6-00_XWT-centroids_train-1982-2018_eval-1982-2018_E13514_XWTs3_Vars-Q850_M-6-7-8-9-10.nc'
    sWTdata = dir_data + 'Clusters13514_HUC6-00_1982-2018_Q850_JJASO'
    NormFact = dir_data + 'Normalization_Factors_IFS_HUC6-00.npz'
    WTsort = [1,2,0]
    re = 2
if RegName == 'NM South':
    # cluster HUC6-03
    sERA_data = dir_data + 'ERA-Interim_PRISM_data13514_HUC6-03_1982-2018_Q850_JJASO.np'
    sCentroidFile = dir_data + 'HUC6-03_XWT-centroids_train-1982-2018_eval-1982-2018_E13514_XWTs3_Vars-Q850_M-6-7-8-9-10.nc'
    sWTdata = dir_data + 'Clusters13514_HUC6-03_1982-2018_Q850_JJASO'
    NormFact = dir_data + 'Normalization_Factors_IFS_HUC6-03.npz'
    WTsort = [0,1,2]
    re = 3

In [41]:
# Load the Centroids
print('Centroids:', sWTdata)
with open(sWTdata, 'rb') as handle:
    npzfile = pickle.load(handle)
WTclusters=npzfile['grClustersFin']['Full']
WTlat=npzfile['LatWT']#; rgrLatT1 = WTlat
WTlon=npzfile['LonWT']#; rgrLonT1 = WTlon
# WTlon[WTlon<0] = WTlon[WTlon<0]+360
WTtime=npzfile['rgdTime']
SpatialSmoothing=npzfile['SpatialSmoothing']
print('centroid shape:', WTlat.shape)

Centroids: /glade/campaign/mmm/c3we/mingge/COEXIST/AZ_S2S_DATA/Clusters13514_1501_1982-2018_Q850_JJASO
centroid shape: (18, 17)


In [42]:
# read ERA-INTERIM coordinates
dir_erai = '/gpfs/fs1/collections/rda/data/ds627.0/ei.oper.an.pl/'
flnm = dir_erai + '200112/ei.oper.an.pl.regn128sc.2001121812'
print(flnm)

ds = xr.open_dataset(flnm, engine="cfgrib", backend_kwargs={'indexpath' :'' })
print(ds)

rgrLonS=ds['longitude'][:]
rgrLatS=ds['latitude'][:]
n_lat_i = rgrLatS.shape[0]
n_lon_i = rgrLonS.shape[0] 
 
rgrLonSF, rgrLatSF = np.meshgrid(rgrLonS, rgrLatS)
rgrLonSF[rgrLonSF>180] = rgrLonSF[rgrLonSF>180]-360

# !!!!  NOTE: reverse Lat: make it from south to North
rgrLatSF = rgrLatSF[::-1] # ERA-Interim lat runs from N to S
 
# get the region of interest
iAddCells= 4 # grid cells added to subregion
iWest=np.argmin(np.abs(WTlon.min() - rgrLonSF[0,:]))-iAddCells
iEast=np.argmin(np.abs(WTlon.max() - rgrLonSF[0,:]))+iAddCells
iNort=np.argmin(np.abs(WTlat.max() - rgrLatSF[:,0]))+iAddCells
iSouth=np.argmin(np.abs(WTlat.min() - rgrLatSF[:,0]))-iAddCells

rgrLonS=rgrLonSF[iSouth:iNort,iWest:iEast]
rgrLatS=rgrLatSF[iSouth:iNort,iWest:iEast]

print('ERA-INTERIM shape:',rgrLonS.shape)

# create gregridding weights
# Remap ECMWF to ERA-Interim
# WTlon.flatten() -> WTlon.flatten()[:,None]
#          (306,) -> (306, 1)
# wts.shape (306, 3)
points=np.array([rgrLonS.flatten(), rgrLatS.flatten()]).transpose()
vtx, wts = interp_weights(points, np.append(WTlon.flatten()[:,None], WTlat.flatten()[:,None], axis=1))

/gpfs/fs1/collections/rda/data/ds627.0/ei.oper.an.pl/200112/ei.oper.an.pl.regn128sc.2001121812
<xarray.Dataset>
Dimensions:        (isobaricInhPa: 37, latitude: 256, longitude: 512)
Coordinates:
    number         int64 ...
    time           datetime64[ns] ...
    step           timedelta64[ns] ...
  * isobaricInhPa  (isobaricInhPa) int64 1000 975 950 925 900 875 ... 7 5 3 2 1
  * latitude       (latitude) float64 89.46 88.77 88.07 ... -88.07 -88.77 -89.46
  * longitude      (longitude) float64 0.0 0.7031 1.406 ... 357.9 358.6 359.3
    valid_time     datetime64[ns] ...
Data variables:
    pv             (isobaricInhPa, latitude, longitude) float32 ...
    z              (isobaricInhPa, latitude, longitude) float32 ...
    t              (isobaricInhPa, latitude, longitude) float32 ...
    q              (isobaricInhPa, latitude, longitude) float32 ...
    w              (isobaricInhPa, latitude, longitude) float32 ...
    vo             (isobaricInhPa, latitude, longitude) float32 ..

#### Calculate Normalization data

In [43]:
year_s = 1979 
year_e = 2018
#year_e = year_s
n_year = year_e - year_s + 1

# there are n_day_s2s = 153 days from June to Oct
dStartDay=datetime(year_s, int(MONTHS[0]), 1,  12)
dStopDay=datetime(year_s, int(MONTHS[-1]), 31, 12)
rgdTimeDD = pd.date_range(dStartDay, end=dStopDay, freq='d')
n_day_s2s = len(rgdTimeDD)
print(n_day_s2s, 'Days in', MONTHS)

dir_o = '/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/WT-frequency/'
dir_q = '/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/Q850/' + RegName.replace(" ", "_")+ '/'

if os.path.isdir(dir_q) != 1:
    subprocess.call(["mkdir","-p", dir_q])
    print('mkdir ', dir_q)
        
data_4d = np.zeros((n_year, n_day_s2s, rgrLonS.shape[0], rgrLonS.shape[1]))

flnm_n = dir_data + 'NormalizationFactors_erai_' + RegName + '.npz'

if os.path.isfile(flnm_n):
    print(flnm_n, 'exist')
    ds_n = np.load(flnm_n, allow_pickle=True)
    Normalize = ds_n['grNormalizationFactors'] 
else:
    for year in range(year_s, year_e+1): 
        is_first = True
        flnm_o = dir_q + 'q850_' + str(year) + '.nc'
        if os.path.isfile(flnm_o):
            print(flnm_o, 'exist')
            with  xr.open_dataset(flnm_o) as ds:
                 data_4d[year-year_s] = ds.q[:,::-1,:]   
        else:
            for month in MONTHS:
                yyyymm_s = str(year*100 + month)
                print(yyyymm_s)
                
                for day in range(1,month_day_dict[month]+1):
                    for hr_s in ['00','06','12','18']:
                        flnm = dir_erai + yyyymm_s + "/ei.oper.an.pl.regn128sc." + yyyymm_s + str(day).zfill(2)+hr_s
                        #print(flnm)
                 
                        ds = xr.open_dataset(flnm, engine="cfgrib", backend_kwargs={'indexpath' :'' })
                        # select 850hPa, crop to Centroid domain, daily mean
                        # Follow original ERA from North to South
                        q850_2d_0 = ds.q.sel(isobaricInhPa=850).isel(latitude=slice(n_lat_i - iNort, n_lat_i-iSouth)
                                                      ,longitude=slice(iWest, iEast))
                        if hr_s == '00':
                            q850_2d = q850_2d_0
                        else:
                            q850_2d = q850_2d + q850_2d_0
                    q850_2d = q850_2d/4    
                    
                    # concatenate daily data 
                    if is_first:
                        q850_3d = q850_2d
                        is_first = False
                    else:
                        q850_3d = xr.concat([q850_3d, q850_2d], dim="time") 
                            
            print(flnm_o)
            q850_3d.to_netcdf(path = flnm_o)  
            data_4d[year-year_s] =q850_3d[:,::-1,:]
    
    # data_4d from South to North
    Mean = np.nanmean(data_4d, axis=(0,1))
    # interpolate to centroid grid   
    Mean_wt = interpolate(Mean.flatten(), vtx, wts).reshape(WTlon.shape[0],WTlat.shape[1]) 

    # reshape Mean by adding back axis=(1,2) because of np.nanmean
    Anomalies = (data_4d - Mean[None,None,:,:])/Mean[None,None,:,:]

    # Convert input scalar to 1d array with at least one dimension
    # in order to call function  PreprocessWTdata
    Normalize = [np.atleast_1d(np.nanmean(Anomalies)), np.atleast_1d(np.nanstd(Anomalies)), Mean_wt]
     
    print('Amomalies', np.nanmean(Anomalies))
    print(flnm_n)
    np.savez(flnm_n, grNormalizationFactors = Normalize)  

153 Days in [6, 7, 8, 9, 10]
/glade/campaign/mmm/c3we/mingge/COEXIST/AZ_S2S_DATA/NormalizationFactors_erai_Arizona West.npz exist


In [44]:
dir_q

'/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/Q850/Arizona_West/'

#### Read ERA-INTERIM data

In [45]:
dir_i = dir_q
dir_o = '/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/WT-frequency/'

if os.path.isdir(dir_o) != 1:
    subprocess.call(["mkdir", "-p", dir_o])
    print('mkdir ', dir_o)
        
for year in range(year_s, year_e + 1):
    flnm_i = dir_q + 'q850_' + str(year) + '.nc'
    print(flnm_i)
    with  xr.open_dataset(flnm_i) as ds:
        DataAct = ds.q[:,::-1,:].values  
                    
        Data_on_centroid_grid = np.zeros((n_day_s2s, WTlon.shape[0], WTlat.shape[1]))
        Data_on_centroid_grid[:] = np.nan

        for tt in range(n_day_s2s):
            valuesi=interpolate(DataAct[tt,:,:].flatten(), vtx, wts)
            Data_on_centroid_grid[tt,:,:] = valuesi.reshape(WTlon.shape[0],WTlat.shape[1])

        # Associiate dta to WTs
        SHAPE = Data_on_centroid_grid.shape
        WT_NMME = np.zeros((SHAPE[0])) 
        WT_NMME[:] = np.nan
   
        if np.isnan(np.nanmean(Data_on_centroid_grid[:,:,:])) == False:
            isNAN = np.isnan(np.nanmean(Data_on_centroid_grid[:,:,:], axis=(1,2)))
            DailyVarsEvalNorm=PreprocessWTdata(Data_on_centroid_grid,               # WT data [time,lat,lon,var]
                            RelAnnom=RelAnnom,                                              # calculate relative anomalies [1-yes; 0-no]
                            SmoothSigma=0,                                                  # Smoothing stddev (Gaussian smoothing)
                            RemoveAnnualCycl=RemoveAnnualCycl,                              # remove annual cycle [1-yes; 0-no]
                            NormalizeData=NormalizeData,                                    # normalize data [1-yes; 0-no]
                            Normalize = Normalize)                                           # predefined mean and std for normalization  

            EucledianDist, Correlation, Manhattan, Chebyshev = EucledianDistance(DailyVarsEvalNorm,
                                                      WTclusters)
            EucledianDist_orig=np.copy(EucledianDist)
            EucledianDist=np.nanmin(EucledianDist,axis=1)
            MinED = np.nanargmin(EucledianDist_orig,axis=1).astype(float)
            MinED[isNAN] = np.nan
            WT_NMME = MinED

        # Re-label the WTs so that: 0=Monsoon, 1=Normal, 2=Dry
        WT_NMME_FIN = np.copy(WT_NMME); WT_NMME_FIN[:] = 999
        for wt in range(len(WTsort)):
            WT_NMME_FIN[WT_NMME == WTsort[wt]] = int(wt)
            
        ### Process the output into monthly mean WT frequencies and save them into a csv file
        WT_MM = np.zeros((5, 3))
        rgiDDmon = [30,31,31,30,31]
        for wt in range(3):
            for mm in range(5):
                WT_MM[mm,wt] = np.sum(WT_NMME_FIN[int(np.sum(rgiDDmon[:mm])):int(np.sum(rgiDDmon[:mm+1]))] == wt, axis=0)    
        
        YYYYMM = [ (year*100+mm) for mm in MONTHS]
        DATA = np.round(WT_MM,2)
        
        grDATA = {}
        grDATA['YYYYMM'] = YYYYMM
        grDATA['dry']  = DATA[:len(YYYYMM),2].astype(float)
        grDATA['normal']  = DATA[:len(YYYYMM),1].astype(float)
        grDATA['monsoon']  = DATA[:len(YYYYMM),0].astype(float)
        df = pd.DataFrame(grDATA)
         
        flnm_o = dir_o + 'ERAI_WT-frequency_'+ RegName.replace(' ','-') + '-'+ str(year) +'.csv'
        print(flnm_o)
        df.to_csv(flnm_o, index=False)
print('finish')

/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/Q850/Arizona_West/q850_1979.nc
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/WT-frequency/ERAI_WT-frequency_Arizona-West-1979.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/Q850/Arizona_West/q850_1980.nc
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/WT-frequency/ERAI_WT-frequency_Arizona-West-1980.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/Q850/Arizona_West/q850_1981.nc
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/WT-frequency/ERAI_WT-frequency_Arizona-West-1981.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/Q850/Arizona_West/q850_1982.nc
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/WT-frequency/ERAI_WT-frequency_Arizona-West-1982.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/Q850/Arizona_West/q850_1983.nc
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/WT-frequency/ERAI_WT-frequency_Arizona-West-1983.csv
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/Q850/Arizona_West/q850_1984.nc
/glade/campaign/mmm/c3we/mingge/COEXIST/ERAI/WT-freque